# **Bronze layer**  - Ingesting data to databricks

In [0]:
import requests, json
from pyspark.sql import SparkSession

#Defining Spark Session
spark=SparkSession.builder.appName("JobETL_Bronze").getOrCreate()

API_KEY = "630f7f2b9c335376c24528a0c6997245ea046c46929b4ead4211322c87bf9064"

roles = ["Data Engineer", "Data Analyst", "Python Developer", "ETL Developer", "Spark Engineer"]
location = "India"
all_jobs=[]

#running a loop over the job roles
for role in roles:
    params={
        "engine":"google_jobs",
        "q":role,
        "location":location,
        "api_key":API_KEY
    }
    res = requests.get("https://serpapi.com/search.json", params=params)

    jobs = res.json().get("jobs_results", [])

    for job in jobs:
        job["search_role"] = role
    all_jobs.extend(jobs)

#bronze_df = spark.read.json(spark.sparkContext.parallelize([json.dumps(job) for job in all_jobs]))

bronze_df = spark.createDataFrame(all_jobs)
bronze_df.write.format("delta").option("mergeSchema",'true').mode("overwrite").saveAsTable("bronze_jobs_raw")

In [0]:
display(bronze_df)

**Silver Layer** - cleaning and structuring data


In [0]:
from pyspark.sql.functions import *

spark=SparkSession.builder.appName("JobETL_Silver").getOrCreate()

#reading bronze layer
df =spark.read.table("bronze_jobs_raw")

#cleaning and structuring
silver_df= df.selectExpr(
    "title",
    "company_name",
    "location",
    "description",
    "job_id",
    "detected_extensions.posted_at as posted_at",
    "search_role"
).dropna(subset=["title","company_name","location"])
 
display(silver_df)


In [0]:
#trimming company name column and make it upper 
silver_df=silver_df.withColumn("company_name",trim(upper(col("company_name"))))
#dropping na in posted at column
silver_df=silver_df.dropna(subset=["posted_at"])

#writing the data to silver table
silver_df.write.format("delta").option("mergeSchema",'true').mode("overwrite").saveAsTable("silver_jobs_raw")
display(silver_df)